# Sentiment Analysis with Logistic Regression

In this activity, you will

- Explore the data to understand the dataset

- We want to know what keyword with positive sentiment (5 star reviews) and neutral sentiment (3 star reviews) and negative sentiment (1 star reviews)

- We also want to know where the model falls short. Analyze the errors that your model makes


## Steps

1) Download data

2) Open
	- How many rows?
	- How many columns? What are they?

3) Preprocess
	- Label 1 star --> negative 3 -->neutral  5--> positive
	- Discard 2 and 4 star reviews

4) Split train test (Think how many)

5) Make feature vector (vectorize -- CountVectorizer)

6) Train logistic regression on training set

7) Evaluate on test set

8) Predict on test set and add a column to test set

     | text | gold standard label | predicted label |
     |------|---------------------|-----------------|

9) Save to Excel

10) Open the prediction file in Excel and find error patterns

In [1]:
!gdown --id 1UpSJkqHhU7RMQC47Ii-6VZuByPYfhLrj

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1UpSJkqHhU7RMQC47Ii-6VZuByPYfhLrj
To: /content/w_review_data.csv
100% 60.4M/60.4M [00:01<00:00, 31.2MB/s]


In [2]:
!head w_review_data.csv

"ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย 
ลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่
ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล 
พะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว
เลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) 

คงไม่ไปซ้ำ แพงเกิน ";3
"อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก ""ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 


In [3]:
# prompt: open  w_review_data.csv use ; as separator

import pandas as pd

# Load the CSV file with ';' as the separator
df = pd.read_csv('w_review_data.csv', sep=';', header=None)

# Now you can work with the DataFrame 'df'
print(df.head())


                                                   0  1
0  ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...  3
1  อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...  4
2  ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...  3
3  รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...  5
4  อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...  5


In [4]:
# Change column names
df.columns = ['text', 'star']


In [5]:
# prompt: Discard reviews with 4 stars and 2 stars

# Discard 2 and 4 star reviews
df = df[~df['star'].isin([2, 4])]


In [6]:
# prompt: Count number of rows again

print(len(df))


19385


In [7]:
# prompt: Create a label: 1 star = negative 3 stars = neutral 5 stars = positive

# Create a function to label the sentiments
def label_sentiment(star_rating):
    if star_rating == 1:
        return 'negative'
    elif star_rating == 3:
        return 'neutral'
    elif star_rating == 5:
        return 'positive'
    else:
        return None  # Handle other star ratings as needed

# Apply the function to create the 'sentiment' column
df['sentiment'] = df['star'].apply(label_sentiment)

# Remove rows where the sentiment is None (2 and 4 star reviews are already removed)
df = df.dropna(subset=['sentiment'])

df.head()


,text,star,sentiment
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,neutral
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,neutral
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,positive
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,positive
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3,neutral


In [8]:
# prompt: Split the data into 70:30 train test

from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame with 'text' and 'sentiment' columns

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42) # random_state for reproducibility

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")


Training set size: 13569
Testing set size: 5816


In [9]:
# prompt: Vectorize the data using pythainlp word_tokenize

!pip install pythainlp

from pythainlp import word_tokenize

# ... (Your existing code) ...

# Assuming 'train_df' and 'test_df' are defined

def tokenize_text(text):
    return word_tokenize(text, engine='newmm')

train_df['tokenized_text'] = train_df['text'].apply(tokenize_text)
test_df['tokenized_text'] = test_df['text'].apply(tokenize_text)

print(train_df.head())
print(test_df.head())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 23.6 MB/s eta 0:00:00
                                                    text  star sentiment  \
5841   ร้านโกวใหญ่ ขายก๋วยเตี๋ยวต้มยำ \nพิกัดร้าน ร้า...     3   neutral   
37117  ร้านอยู่บนถนนวงแหวนกาญจนาภิเษก บางใหญ่\nร้านให...     3   neutral   
26320  หลายวันก่อน ผมมีโอกาสแวะมาทำธุระที่ จ.กำแพงเพช...     3   neutral   
35495  ⚓ ที่ตั้ง : ตลาดรวมทรัพย์ ณ ถนนอโศกมนตรี โซนอา...     3   neutral   
36836  set cheesy tonkatsu-com ก็โอเคน้าาา เราว่ามันๆ...     3   neutral   

                                          tokenized_text  
5841   [ร้าน, โก, ว, ใหญ่,  , ขาย, ก๋วยเตี๋ยว, ต้มยำ,...  
37117  [ร้าน, อยู่, บน, ถนน, วงแหวน, กาญจนาภิเษก,  , ...  
26320  [หลาย, วันก่อน,  , ผม, มีโอกาส, แวะ, มา, ทำ, ธ...  
35495  [⚓,  , ที่ตั้ง,  , :,  , ตลาด, รวม, ทรัพย์,  ,...  
36836  [set,  , cheesy,  , tonkatsu-com,  , ก็, โอเค,...  
                                                    text  star sentiment  \
36642  อาหารอีสานมีให้เลือกหลายเมน

In [10]:
# prompt: Use CountVectorize to vectorize on 'tokenized_text' column

from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'train_df' has a 'tokenized_text' column
train_df['tokenized_text_joined'] = train_df['tokenized_text'].apply(lambda tokens: ' '.join(tokens))

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_vec = vectorizer.fit_transform(train_df['tokenized_text_joined'])

# Transform the test data using the same vectorizer
test_df['tokenized_text_joined'] = test_df['tokenized_text'].apply(lambda tokens: ' '.join(tokens))
X_test_vec = vectorizer.transform(test_df['tokenized_text_joined'])

print(X_train_vec.shape)
X_test_vec.shape


(13569, 17661)


(5816, 17661)

In [11]:
# prompt: Train logistic regression and evaluate the model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increased max_iter
model.fit(X_train_vec, train_df['sentiment'])

# Predict on the test set
y_pred = model.predict(X_test_vec)

# Evaluate the model
print(classification_report(test_df['sentiment'], y_pred))

# Add predictions to the test DataFrame
test_df['predicted_sentiment'] = y_pred

# Display the first few rows of the test DataFrame with predictions
print(test_df[['text', 'sentiment', 'predicted_sentiment']].head())

# Save the test DataFrame with predictions to an Excel file
test_df[['text', 'sentiment', 'predicted_sentiment']].to_excel('predictions.xlsx', index=False)


              precision    recall  f1-score   support

    negative       0.44      0.27      0.33       116
     neutral       0.80      0.82      0.81      3667
    positive       0.68      0.66      0.67      2033

    accuracy                           0.76      5816
   macro avg       0.64      0.58      0.61      5816
weighted avg       0.75      0.76      0.75      5816

                                                    text sentiment  \
36642  อาหารอีสานมีให้เลือกหลายเมนูเลยค่ะ\nรสชาติค่อน...   neutral   
7660   วันนี้มาเดินซื้อของที่แกรนด์พระราม 9 วันหยุดยา...   neutral   
12494  เดินไปหาอะไรทานแถวpaseoวนไปวนมาก็จบด้วยร้านyay...   neutral   
5813   ร้านนี้เป็นร้านสลัด เลือกได้ตามใจ แบบ take hom...   neutral   
23544  ร้านนี้ทานมาตั้งแต่เด็กๆ ที่บ้านชอบมาทานบะหมี่...  positive   

      predicted_sentiment  
36642             neutral  
7660              neutral  
12494             neutral  
5813              neutral  
23544            positive  
